In [1]:
import numpy as np
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import (Input, Dense, Concatenate)
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from utils import *

Using TensorFlow backend.


### Loading Processed Data Set

Loading of the pre-processed data set with all new features and all rows.

In [2]:
processed_wdch = pd.read_csv('data/final_processed.csv')

In [3]:
len(processed_wdch)

1017199

In [4]:
processed_wdch.columns

Index(['trajectory_id', 'x_entry', 'y_entry', 'x_exit', 'y_exit', 'test',
       'entry_in_town', 'sum_entry_in_town', 'sum_exit_in_town',
       'distance_traveled', 'sum_moved_into_town', 'sum_moved_outof_town',
       'time_difference', 'avg_time_difference', 'calculated_velocity',
       'time_entry_seconds', 'time_exit_seconds', 'distance_from_city_center',
       'trajectory_num', 'exit_in_town', 'percentage_change_square'],
      dtype='object')

### Splitting the data set in preparation for modelling

Since we know that all trajectories with no time elapsed do not change location, we will seperate these instances and upload the starting location as the prediction for these later:

In [7]:
to_submit = processed_wdch[['trajectory_id','entry_in_town']][(processed_wdch['time_difference']==0) & 
                                                         (processed_wdch['test']==1)&(processed_wdch['x_exit'].isnull())]


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


The rest of the dataset (trajectories where there is some time elapsed) will be split into train and test for modelling:

In [ ]:
processedScaled = processed_wdch
train_r = processedScaled[(processedScaled['time_difference']!=0) &  (~processedScaled['x_exit'].isnull()) ]
test_r = processedScaled[(processedScaled['time_difference']!=0) &  (processedScaled['x_exit'].isnull()) ]
test_r.drop(['x_exit', 'y_exit'], axis=1, inplace=True)
train_r.drop(['x_exit', 'y_exit'], axis=1, inplace=True)
Xc = train_r.drop(['trajectory_id','exit_in_town'],axis =1)
yc = train_r['exit_in_town'][~train_r['exit_in_town'].isnull()]
test = test_r.drop(['trajectory_id','exit_in_town'],axis =1)

### Denoising Autoencoder 

In [12]:
Scaler = MinMaxScaler()
X_scale = Scaler.fit_transform(Xc)
test_scale = Scaler.transform(test)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [13]:
noise_factor = 0.4
x_train_noisy = X_scale + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_scale.shape)
x_test_noisy = test_scale + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=test_scale.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)


In [16]:
def DEEP_DAE(features_shape, act='relu'):
 
    # Input
    x = Input(name='inputs', shape=features_shape, dtype='float32')
    o = x
    
    # Encoder / Decoder
    o = Dense(128, activation=act, name='dense1')(o)
    o = Dense(64, activation=act, name='dense2')(o)
    o = Dense(32, activation=act, name='dense3')(o)
    o = Dense(64, activation=act, name='dense4')(o)
    o = Dense(128, activation=act, name='dense5')(o)
    dec = Dense(17, activation='linear', name='dense_dec')(o)
    
    # Print network summary
    Model(inputs=x, outputs=dec).summary()
    
    return Model(inputs=x, outputs=dec)

In [17]:
x_feat_train = np.concatenate((X_scale, test_scale), axis=0)
x_feat_train_noisy = np.concatenate((x_train_noisy, x_test_noisy), axis=0)

In [18]:
x_feat_train.shape

(476266, 17)

In [19]:
batch_size = 128
epochs = 300
             
autoenc = DEEP_DAE((17,))
autoenc.compile(optimizer='adadelta', loss='binary_crossentropy')

autoenc.fit(x_feat_train_noisy, x_feat_train, epochs=epochs, 
            batch_size=batch_size, shuffle=True)

decoded_imgs = autoenc.predict(x_feat_train_noisy)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 17)                0         
_________________________________________________________________
dense1 (Dense)               (None, 128)               2304      
_________________________________________________________________
dense2 (Dense)               (None, 64)                8256      
_________________________________________________________________
dense3 (Dense)               (None, 32)                2080      
_________________________________________________________________
dense4 (Dense)               (None, 64)                2112      
_________________________________________________________________
dense5 (Dense)               (None, 128)               8320      
_________________________________________________________________
dens

476266/476266 [==============================] - 12s 26us/step - loss: 1.1736
Epoch 160/300
476266/476266 [==============================] - 13s 28us/step - loss: 0.9028
Epoch 161/300
476266/476266 [==============================] - 11s 23us/step - loss: 0.8579
Epoch 162/300
476266/476266 [==============================] - 11s 23us/step - loss: 0.8570
Epoch 163/300
476266/476266 [==============================] - 11s 24us/step - loss: 0.8568
Epoch 164/300
476266/476266 [==============================] - 11s 23us/step - loss: 0.8574
Epoch 165/300
476266/476266 [==============================] - 13s 28us/step - loss: 0.8584
Epoch 166/300
476266/476266 [==============================] - 10s 21us/step - loss: 0.8579
Epoch 167/300
476266/476266 [==============================] - 10s 21us/step - loss: 0.8575
Epoch 168/300
476266/476266 [==============================] - 10s 21us/step - loss: 0.8565
Epoch 169/300
476266/476266 [==============================] - 10s 21us/step - loss: 0.8563
Ep

In [30]:

def FEATURES(model):
    input_ = model.get_layer('inputs').input
#     feat1 = model.get_layer('dense2').output
    feat = model.get_layer('dense3').output
#     feat3 = model.get_layer('dense4').output
#     feat = Concatenate(name='concat')([feat1, feat2, feat3])
    model = Model(inputs=[input_],
                      outputs=[feat])
    return model

_model = FEATURES(autoenc)
features_train = _model.predict(X_scale)
features_test = _model.predict(test_scale)
print(features_train.shape, ' train samples shape')
print(features_test.shape, ' test samples shape')



(459592, 32)  train samples shape
(16674, 32)  test samples shape


# Principal Component Analysis

Given the increase in the number of features generated by the Denoising Autoencoder we decided to run PCA on the features before including them as part of our model:

In [31]:
from sklearn.decomposition import PCA

In [33]:
pca = PCA(n_components=5)
pca.fit(features_train)

PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [34]:
pca.explained_variance_ratio_

array([0.65641123, 0.19615542, 0.08007728, 0.02023567, 0.01550091])

In [35]:
features_train.shape

(459592, 32)

In [42]:
reduced_train = pca.transform(features_train)
reduced_test = pca.transform(features_test)

In [43]:
new_features_train  = pd.DataFrame(reduced_train , index = Xc.index)

In [44]:
new_features_test  = pd.DataFrame(reduced_test , index =test.index)

In [45]:
new_features_test.reset_index(inplace = True)

In [46]:
new_train = pd.concat([Xc,new_features_train],axis = 1 )

In [47]:
new_test = pd.concat([test,new_features_test],axis = 1 )

### Modelling

With the dataset including the new features we will run a XGBoost Classifier, which has proved to be the best algorithm to classify the target for this problem during previous attempts.

In [220]:
xgb = XGBClassifier(colsample_bytree= 0.75,
 learning_rate= 0.01,
 max_depth= 95,
 min_child_weight= 9,
 n_estimators= 350,
 random_state= 6666,
 subsample=0.75, n_jobs = -1,verbose = True)
xgb.fit(new_train,yc)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.75, gamma=10, learning_rate=0.001,
       max_delta_step=0, max_depth=100, min_child_weight=3, missing=None,
       n_estimators=400, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=6666, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.75, verbose=True)

In [222]:
xgb_f_pred = xgb.predict(new_test)

In [221]:
feature_importances_xgUK = pd.DataFrame(xgb.feature_importances_,
                                   index =new_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances_xgUK 

,importance
x_entry,0.100333
time_difference,0.094763
distance_from_city_center,0.072369
y_entry,0.065420
percentage_change_square,0.063177
time_entry_seconds,0.058564
trajectory_num,0.057829
avg_time_difference,0.054740
time_exit_seconds,0.053257
0,0.048653


Now that we have predictions for the test set, this needs to be joined with the trajetories where there was no time elapsed.

In [223]:
predsi = xgb_f_pred

In [224]:
test_pred = test_r.copy()
test_pred['pred'] = predsi
predsi= test_pred[['trajectory_id','pred']]
predsi.columns = ['id','target']
to_submit.columns = ['id','target']
preds = pd.concat([predsi,to_submit],axis=0)

Save predictions as CSV format to submit:

In [226]:
preds.to_csv('59_88.7_GS1.csv', index=False)

In [225]:
preds.sum()

id        traj_00032f51796fd5437b238e3a9823d13d_31_5traj...
target                                                 8367
dtype: object